In [36]:
import os
from contextlib import ExitStack

import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
import math

import torch
import torchtext
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from torchmetrics.regression import MeanSquaredError
from torchmetrics.regression import R2Score

In [37]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

#get device that we are using
#torch.cuda.device_count()
#torch.cuda.current_device()
#torch.cuda.get_device_name(0)

Using cuda device


In [62]:
#data processing
with ExitStack() as stack:
    data = sio.loadmat('Random.mat')

    # Extract the data
    C0 = np.array(data['C0'])
    Seq = np.array(data['Seq'])
    SS = np.array(data['SS'])
Seq
#data = np.concatenate((Seq, C0))

array(['TCCGCACTTATGTACTGTGCTGAGATATAGTAGATTCTGCGTGTGATCGA',
       'GTCTCGATCCACCGCTAGTAGTAAGACAACAGGGCTGCCTGGCTTCAACT',
       'TGTTGGCCCAAGCTACTTCCGTTTACCAGAACCACAGTGTTAAGGGCTTC', ...,
       'AACCTTGCTAGTGAACTGTGCCTCGTCGCGGTAATCCACGGAATATGTTG',
       'TCTGGGGGTGGTCTCGTTCGAGTTTCCGGAGAATAGACTCGGCGGGTCCA',
       'AGGATCCCCCCGACTAGTACTGAAGTAACCAGCTATTCCTGTTTGGCAGC'], dtype='<U50')

In [39]:
#split data into test train

x_train, x_test, y_train, y_test = train_test_split(
    Seq, C0, test_size=0.1, random_state=42
)

In [80]:
#adapted from https://towardsdatascience.com/custom-datasets-in-pytorch-part-2-text-machine-translation-71c41a3e994e

class Vocabulary:
    '''
    __init__ is called when object is initiated, to initiate vocab dictionaries
    '''
    def __init__(self):
        #initiate index to token dict
        # PAD: padding to match sentence lengths
        # SOS: start token
        # EOS: end token
        # UNK: words not found in the vocab 
        self.itos = {0: '<PAD>', 1:'<SOS>', 2:'<EOS>', 3:'<UNK>'}
        #initiate token to index dict
        self.stoi = {k:j for j,k in self.itos.items()} 
    
    '''
    __len__ used by dataloader to create batches (maybe need?)
    '''
    def __len__(self):
        return len(self.itos)
        
    '''tokenizer converts each character in RNA sequence to an element in a list'''
    @staticmethod
    def tokenizer(text, group_size):
        #split sequences into sizes of the group size
        tok_lower = text.lower()
        return [tok_lower[i:i+group_size] for i in range(0, len(tok_lower), group_size)]
    
    #increase vocabulary size: go up to 4 depending on which is most popular --> divides evenly as well
    '''build the vocab: map index to string, string to index'''
    def build_vocab(self):
        idx = 4 #start at 4 b/c 0 to 3 are taken
        #5 letter residues
        residues = ["a", "t", "c", "g"]
        self.group_size = 5
        for res1 in residues:
            for res2 in residues:
                for res3 in residues:
                    for res4 in residues:
                        for res5 in residues:
                            group = res1+res2+res3+res4+res5
                            self.stoi[group] = idx
                            self.itos[idx] = group
                            idx+=1

    def numericalize(self, text):
        tokenized_text = self.tokenizer(text, self.group_size)
        numericalized_text = []
        for token in tokenized_text:
            if token in self.stoi.keys():
                numericalized_text.append(self.stoi[token])
            else:
                numericalized_text.append(self.stoi['<UNK>'])
        return numericalized_text

In [81]:
v_test = Vocabulary()
v_test.build_vocab()
v_test.__len__()

1028

In [82]:
#####################
#Build Train Dataset#
#####################

class Train_Dataset(Dataset):
    '''
    Variables
    x_train: training sequences
    y_train: training flexibility index
    transform: if we want to add augmentation
    '''

    def __init__(self, x_train, y_train, transform=None):
        self.x_train = x_train
        self.y_train = y_train
        self.transform = transform

        #create vocabulary (only for x data):
        self.source_vocab = Vocabulary()
        self.source_vocab.build_vocab()
    
    def __len__(self):
        return len(self.x_train)
    
    '''__getitem___ runs on 1 item at a time'''

    def __getitem__(self, index):
        source_text = self.x_train[index]
        if self.transform is not None:
            source_text = self.transform(source_text)
        
        #numericalize texts
        numericalized_source = [self.source_vocab.stoi["<SOS>"]]
        numericalized_source += self.source_vocab.numericalize(source_text)
        numericalized_source.append(self.source_vocab.stoi["<EOS>"])

        target = self.y_train[index][0]

        return torch.tensor(numericalized_source), torch.tensor(target, dtype=torch.float)

In [83]:
class Validation_Dataset(Dataset):
    '''
    Variables
    x_train: training sequences
    y_train: training flexibility index
    transform: if we want to add augmentation
    '''

    def __init__(self, train_dataset, x_test, y_test, transform=None):
        self.train_dataset = train_dataset
        self.x_test = x_test
        self.y_test = y_test
        self.transform = transform
    
    def __len__(self):
        return len(self.x_test)
    
    '''__getitem___ runs on 1 item at a time'''

    def __getitem__(self, index):
        source_text = self.x_test[index]
        if self.transform is not None:
            source_text = self.transform(source_text)
        
        #numericalize texts
        numericalized_source = [self.train_dataset.source_vocab.stoi["<SOS>"]]
        numericalized_source += self.train_dataset.source_vocab.numericalize(source_text)
        numericalized_source.append(self.train_dataset.source_vocab.stoi["<EOS>"])

        target = self.y_test[index][0]

        return torch.tensor(numericalized_source), torch.tensor(target, dtype=torch.float)

In [84]:
#########
#Collate#
#########

'''Adds padding (shouldn't need the mycollate function for this application)'''
class MyCollate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx

    #__call__: runs by default when MyCollate is created
    def __call__(self, batch):
        source = [item[0] for item in batch]
        source = pad_sequence(source, batch_first=False, padding_value=self.pad_idx)

In [85]:
#loader functions
def get_train_loader(dataset, batch_size, num_workers=0, shuffle=True, pin_memory=True): #increase num_workers according to CPU
    #get pad_idx for collate fn
    pad_idx = dataset.source_vocab.stoi['<PAD>']
    #print(pad_idx)
    #define loader
    loader = DataLoader(dataset, batch_size = batch_size, num_workers = num_workers,
                        shuffle=shuffle,
                       pin_memory=pin_memory) 
    return loader

def get_valid_loader(dataset, train_dataset, batch_size, num_workers=0, shuffle=True, pin_memory=True):
    pad_idx = train_dataset.source_vocab.stoi['<PAD>']
    loader = DataLoader(dataset, batch_size = batch_size, num_workers = num_workers,
                        shuffle=shuffle,
                       pin_memory=pin_memory)
    return loader

In [86]:
#transformer adapted from https://n8henrie.com/2021/08/writing-a-transformer-classifier-in-pytorch/

class PositionalEncoding(nn.Module):
    '''https://pytorch.org/tutorials/beginner/transformer_tutorial.html'''

    #vocab size is 8 i think
    
    #nvm i think vocab size needs to be larger than seq size: this way it can shrink down to the seq size
    #positional encoder encodes a unique function for each index and position in the text
    
    def __init__(self, d_model, vocab_size=5000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(vocab_size, d_model)
        position = torch.arange(0, vocab_size, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float()
            * (-math.log(10000.0)/d_model)
        )
        pe[:, 0::2] = torch.sin(position*div_term)
        pe[:, 1::2] = torch.cos(position*div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)
    
    def forward(self, x):
        x = x + self.pe[:, : x.size(1), :]
        return self.dropout(x)

In [87]:
class TransformerRegressionModel(nn.Module):
    '''text regression model using a transformer'''

    def __init__(
            self,
            vocab_size,
            d_model,
            n_head,
            dim_feedforward,
            num_layers,
            dropout,
            classifier_dropout,
            activation = "relu"
    ):
        
        super().__init__()
        self.model_type = "Transformer"
        self.vocab_size = vocab_size
        assert d_model % n_head == 0 #assert that the number of attention heads divides into the size of the embedding (d_model)
        
        #may need to make vocab_size a bit bigger (same as encoding) for the embedding and encoding to work
        self.emb = nn.Embedding(vocab_size, d_model) #embedding of size d_model
        self.pos_encoder = PositionalEncoding(d_model, vocab_size)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_head, dim_feedforward=dim_feedforward, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        self.regressor = nn.Linear(d_model, 1)
        self.d_model = d_model
    
    def forward(self, x):
        x=self.emb(x) * math.sqrt(self.d_model)
        x=self.pos_encoder(x)
        x=self.transformer_encoder(x)
        x=x.mean(dim=1) #taking the mean along the sequence size dimension is a common way to process transformer output
        x=self.regressor(x)
        x=x.squeeze(-1)
        return x

In [88]:
train_data = Train_Dataset(x_train, y_train)
print(x_train[0], y_train[0])
train_data[0]

print(train_data.source_vocab.stoi)

CAGCCTTCTTACTCTTACGCCCAGGGCATACGGGGGGTTCCCGATCGATG [0.27438]
{'<PAD>': 0, '<SOS>': 1, '<EOS>': 2, '<UNK>': 3, 'aaaaa': 4, 'aaaat': 5, 'aaaac': 6, 'aaaag': 7, 'aaata': 8, 'aaatt': 9, 'aaatc': 10, 'aaatg': 11, 'aaaca': 12, 'aaact': 13, 'aaacc': 14, 'aaacg': 15, 'aaaga': 16, 'aaagt': 17, 'aaagc': 18, 'aaagg': 19, 'aataa': 20, 'aatat': 21, 'aatac': 22, 'aatag': 23, 'aatta': 24, 'aattt': 25, 'aattc': 26, 'aattg': 27, 'aatca': 28, 'aatct': 29, 'aatcc': 30, 'aatcg': 31, 'aatga': 32, 'aatgt': 33, 'aatgc': 34, 'aatgg': 35, 'aacaa': 36, 'aacat': 37, 'aacac': 38, 'aacag': 39, 'aacta': 40, 'aactt': 41, 'aactc': 42, 'aactg': 43, 'aacca': 44, 'aacct': 45, 'aaccc': 46, 'aaccg': 47, 'aacga': 48, 'aacgt': 49, 'aacgc': 50, 'aacgg': 51, 'aagaa': 52, 'aagat': 53, 'aagac': 54, 'aagag': 55, 'aagta': 56, 'aagtt': 57, 'aagtc': 58, 'aagtg': 59, 'aagca': 60, 'aagct': 61, 'aagcc': 62, 'aagcg': 63, 'aagga': 64, 'aaggt': 65, 'aaggc': 66, 'aaggg': 67, 'ataaa': 68, 'ataat': 69, 'ataac': 70, 'ataag': 71, 'atata': 72,

In [90]:
test_data = Validation_Dataset(train_data, x_test, y_test)
print(x_test[0], y_test[0], len(x_test[0]))
test_data[0]

CGCGGCTGGAAGTAACCTACTCAAACATTGTGCATCCGCCTGGCTCGGGT [-0.56108] 50


(tensor([  1, 755, 640, 212, 662, 388, 539, 485, 702, 509, 769,   2]),
 tensor(-0.5611))

In [91]:
train_loader = get_train_loader(train_data, 32)
source = next(iter(train_loader))[0]
flex = next(iter(train_loader))[1]

In [92]:
test_loader = get_valid_loader(test_data, train_data, 32)

In [93]:
vocab_size = train_data.source_vocab.__len__()
d_model = 100
n_head = 5
dim_feedforward = 2048
num_layers = 6
dropout = 0.1
classifier_dropout = 0

#create the model and then move it to the GPU
model = TransformerRegressionModel(vocab_size, d_model, n_head, dim_feedforward, num_layers, dropout, classifier_dropout).to(device)

In [94]:
def accuracy(predictions, target, accuracy_threshold):
    num_correct = 0
    for i in range(0, len(predictions)):
        if abs(predictions[0].item() - target[0].item()) <= target[0].item()*accuracy_threshold:
            num_correct+=1
    return num_correct

In [97]:
import time
import copy

criterion = nn.MSELoss()
lr = 0.001
optimizer = torch.optim.SGD((p for p in model.parameters() if p.requires_grad), lr)

torch.manual_seed(0)

def train(model: nn.Module):
    model.train()
    epoch_train_loss = 0

    total_num_correct = 0
    #begin batching
    for idx, batch in enumerate(iter(train_loader)):
        predictions = model(batch[0].to(device))
        scores = batch[1].to(device)

        loss = criterion(predictions, scores)

        #can add accuracy code, e.g. threshold within 10%; MSE is also a good indicator of correctness
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        epoch_train_loss += loss.item()
        #epoch_train_loss /= len(train_data)
        
        if idx == 349:
            print(f"{predictions=} | {scores=}")
    
        batch_training_accuracy = accuracy(predictions, scores, 0.5)
        total_num_correct += batch_training_accuracy

    print(f"{epoch_train_loss=}")
    train_accuracy = total_num_correct/len(train_data)
    print(f"{train_accuracy=}")

def evaluate(model: nn.Module):
    model.eval()
    epoch_test_loss = 0
    total_num_correct = 0
    
    with torch.no_grad():
        for idx, batch in enumerate(iter(test_loader)):
            output = model(batch[0].to(device))
            scores = batch[1].to(device)

            test_loss = criterion(output, scores)
            epoch_test_loss += test_loss.item()
            #epoch_test_loss /= len(test_data)

            batch_test_accuracy = accuracy(output, scores, 0.1)
            total_num_correct += batch_test_accuracy

    print(f"{epoch_test_loss=}")
    test_accuracy = total_num_correct/len(test_data)
    print(f"{test_accuracy=}")

In [98]:
#train the model!
epochs = 50

print("starting")
for epoch in range(1, epochs + 1):
    print(f"{epoch=}")
    epoch_start_time = time.time()
    train(model)
    evaluate(model)
    elapsed = time.time() - epoch_start_time
    print(f"{elapsed=}")

starting
epoch=1
predictions=tensor([-0.1343, -0.1215, -0.0976, -0.1399, -0.1708, -0.1691, -0.2070, -0.1131,
        -0.2253, -0.1661, -0.1640, -0.1372, -0.0976, -0.2061, -0.1270, -0.2098,
        -0.1038, -0.1724, -0.1684, -0.1350, -0.1007, -0.1464, -0.1200, -0.1080,
        -0.0992, -0.1161, -0.0778, -0.1527, -0.0884, -0.1370, -0.1472, -0.1156],
       device='cuda:0', grad_fn=<SqueezeBackward1>) | scores=tensor([ 0.1185, -0.0751,  0.5645,  0.2053,  0.6346,  0.8052, -0.3633, -0.0565,
         0.0566, -0.2292, -0.5742,  0.4107, -0.2909,  0.1708, -0.3644,  0.1310,
         0.0635, -0.3925, -0.0120,  1.5859, -0.4495,  0.0812, -0.2380,  0.1042,
        -0.2734,  0.1305,  0.2990, -0.2672, -0.3678, -0.1551, -0.3179, -0.1485],
       device='cuda:0')
epoch_train_loss=100.43332381546497
train_accuracy=0.0
epoch_test_loss=11.508541017770767
test_accuracy=0.0
elapsed=10.960482120513916
epoch=2
predictions=tensor([-0.2019, -0.1685, -0.1707, -0.1264, -0.1669, -0.2020, -0.1959, -0.1504,
        -

KeyboardInterrupt: 

In [ ]:
from torchtext.datasets import IMDB

train_iter = iter(IMDB(split='train'))
next(train_iter)

(1,
 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far betwee

In [29]:
for idx, batch in enumerate(iter(train_loader)):
   print(idx, ":", batch[1].size())

#for target in next(iter(train_loader))[1]:
#    print(target.item())

0 : torch.Size([32])
1 : torch.Size([32])
2 : torch.Size([32])
3 : torch.Size([32])
4 : torch.Size([32])
5 : torch.Size([32])
6 : torch.Size([32])
7 : torch.Size([32])
8 : torch.Size([32])
9 : torch.Size([32])
10 : torch.Size([32])
11 : torch.Size([32])
12 : torch.Size([32])
13 : torch.Size([32])
14 : torch.Size([32])
15 : torch.Size([32])
16 : torch.Size([32])
17 : torch.Size([32])
18 : torch.Size([32])
19 : torch.Size([32])
20 : torch.Size([32])
21 : torch.Size([32])
22 : torch.Size([32])
23 : torch.Size([32])
24 : torch.Size([32])
25 : torch.Size([32])
26 : torch.Size([32])
27 : torch.Size([32])
28 : torch.Size([32])
29 : torch.Size([32])
30 : torch.Size([32])
31 : torch.Size([32])
32 : torch.Size([32])
33 : torch.Size([32])
34 : torch.Size([32])
35 : torch.Size([32])
36 : torch.Size([32])
37 : torch.Size([32])
38 : torch.Size([32])
39 : torch.Size([32])
40 : torch.Size([32])
41 : torch.Size([32])
42 : torch.Size([32])
43 : torch.Size([32])
44 : torch.Size([32])
45 : torch.Size([32]